In [27]:
import numpy as np
import pandas as pd

In [28]:
# data preparation
admissions = pd.read_csv('backpropagation_implementation.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
n_records, n_features = features.shape

# n_records x 6
print('Feature values:')
print(features[0:5].values)
print()

# 1 x n_records
print('Target values:')
print(targets[0:5].values)

Feature values:
[[-0.06665712  0.28930534  0.          1.          0.          0.        ]
 [ 0.62588442  1.44547565  0.          1.          0.          0.        ]
 [ 1.83783211  1.60313523  0.          0.          1.          0.        ]
 [ 1.31842596 -0.13112022  0.          0.          0.          1.        ]
 [-0.06665712 -1.20846073  0.          1.          0.          0.        ]]

Target values:
[0 0 1 0 0]


In [29]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [30]:
# Hyperparameters
np.random.seed(42)
n_hidden = 3  # number of hidden units
epochs = 500
learnrate = 0.5

In [31]:
last_loss = None

# Initialize weights
# 6 x 3
weights_input_hidden = np.random.normal(scale=1 / n_features ** -.5,
                                        size=(n_features, n_hidden))
# 1 x 3
weights_hidden_output = np.random.normal(scale=1 / n_features ** -.5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    # x: 1 x 6
    # y: 1
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate error gradient in output unit
        output_error = error * output * (1 - output)

        # TODO: propagate errors to hidden layer
        hidden_error = np.dot(output_error, weights_hidden_output) * hidden_output * (1 - hidden_output)

        # TODO: Update the change in weights
        del_w_hidden_output += output_error * hidden_output
        del_w_input_hidden += hidden_error * x[:, None]

    # TODO: Update weights
    weights_input_hidden += learnrate * del_w_input_hidden / n_records
    weights_hidden_output += learnrate * del_w_hidden_output / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output, weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.3182750870385583
Train loss:  0.31765931344802173
Train loss:  0.31689612476142887
Train loss:  0.316194424075769
Train loss:  0.3155242579527855
Train loss:  0.31479691939207416
Train loss:  0.31395740858119575
Train loss:  0.312972699036571
Train loss:  0.31182033925725555
Train loss:  0.3104845582488786
Prediction accuracy: 0.725


Input: $X$

HiddenInput: $W_{ih} X$

HiddenOutput: $y_h = f(W_{ih} X)$

FinalInput: $W_{ho} f(W_{ih} X)$

FinalOutput: $\hat{y} = f(W_{ho} f(W_{ih} x))$

FinalOutputError: $e = y - \hat{y}$

FinalOutputErrorGradient: $\delta^o = e \times \hat{y}'$

HiddenOutputErrorGraident: $\delta^h = \delta^o \times W_{ho} \times y_h'$

OutputWeightStep: $\Delta W = \Delta W + \delta y_h$

HiddenWeightStep: $\Delta w_h = \Delta w_h + \delta^h X$

OutputWeightUpdate: $W = W + \eta \Delta W / N$

HiddenWeightUpdate: $w_h = w_h + \eta \Delta w_h / N$